# Obtaining metrics

## Dataset: GTSRB

## Output:

- LSA values
    ```python
    "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/lsa_values_1.npy"
    ```
- DSA values
    ```python
    "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/dsa_values_1.npy"
    ```
- Random values
    ```python
    "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/SA_values/gtsrb_random_values_e2.npy"
    ```

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model
from importlib import reload  


In [2]:
pwd

'C:\\Users\\fjdur\\Documents\\upc-july\\notebooks'

In [3]:
cd ../../utils/

C:\Users\fjdur\Documents\utils


In [4]:
# utils for project
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [5]:
pwd

'C:\\Users\\fjdur\\Documents\\utils'

In [6]:
from importlib import reload  
utils = reload(utils)

2.5.0
keras
2.5.0


In [7]:
cd ../MNIST project/

C:\Users\fjdur\Documents\MNIST project


In [8]:
dataset = "mnist"

In [9]:
x_train,y_train = utils.get_data("mnist","Train")
x_val,y_val = utils.get_data("mnist","Val")
x_test,y_test = utils.get_data("mnist","Test")

In [10]:
x_train_and_adversary,y_train_and_adversary = utils.get_data("mnist","Train_and_adversary")

In [11]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)
print("---")
print(x_train[0].shape)
print(x_train_and_adversary.shape)
print(y_train_and_adversary.shape)

(48999, 28, 28, 1)
(48999, 10)
(14000, 28, 28, 1)
(14000, 10)
(7001, 28, 28, 1)
(7001, 10)
---
(28, 28, 1)
(55998, 28, 28, 1)
(55998, 10)


In [12]:
num_classes = y_train_and_adversary.shape[-1]
print(num_classes)

10


In [13]:
#here editing


In [14]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6999
6999


In [15]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data("mnist","Test_adversarial") 

In [16]:
print(x_test_and_adversary.shape)
print(y_test_and_adversary.shape)

(14000, 28, 28, 1)
(14000, 10)


In [17]:
x_adversary_test_fgsm = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test_fgsm))
y_adversary_test_fgsm = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test_fgsm))

6999
6999


In [15]:
#x_test_and_adversary_jsma,y_test_and_adversary_jsma = utils.get_adversarial_data('Test_jsma') 

## ----

In [16]:
# Original model 
#model_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/models/tf_model_25-06"

In [17]:
#model_original = utils.My_model(True, model_dir)

In [18]:
# Loading model already trained
model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
model = keras.models.load_model(model_dir)

In [19]:
model.evaluate(x_test,y_test)

219/219 [==============================] - 17s 14ms/step - loss: 0.0366 - accuracy: 0.9894


[0.036614201962947845, 0.9894300699234009]

## Obtaining new LSA and DSA values 
- Required module 'deepimportance_code_release'

In [20]:
cd "D:deepimportance_code_release"

D:\deepimportance_code_release


In [21]:
# from 07_notebook
from utils import filter_val_set, get_trainable_layers
from utils import generate_adversarial, filter_correct_classifications
from coverages.idc import ImportanceDrivenCoverage
from coverages.neuron_cov import NeuronCoverage
from coverages.tkn import DeepGaugeLayerLevelCoverage
from coverages.kmn import DeepGaugePercentCoverage
from coverages.ss import SSCover
from coverages.sa import SurpriseAdequacy


In [22]:
trainable_layers = get_trainable_layers(model)
non_trainable_layers = list(set(range(len(model.layers))) - set(trainable_layers))
print('Trainable layers: ' + str(trainable_layers))
print('Non trainable layers: ' + str(non_trainable_layers))

Trainable layers: [0, 1, 5, 7]
Non trainable layers: [2, 3, 4, 6]


In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [24]:
subject_layer = trainable_layers[-2]
layer_names = [model.layers[-3].name]
upper_bound = 2000
layer_names

['dense_2']

In [25]:
subject_layer

5

In [28]:
from coverages import sa
from datetime import datetime

In [29]:
pwd

'D:\\deepimportance_code_release'

In [34]:
start_time = datetime.now()
#surprise of all inputs wrt to the original inputs
target_lsa = sa.fetch_lsa(model, x_train, x_train_and_adversary,
                                   "conf_1_jul", layer_names,
                                   num_classes, True,
                                   1e-05, './sa_'+dataset, "model_31_jul")


end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

[train] Model serving
8
383/383 [==============================] - 19s 48ms/step
[train] Processing ATs
Layer: dense_2
train ATs is saved at ./sa_mnist\model_31_jul_train_dense_2_ats.npy
[conf_1_jul] Model serving
8
438/438 [==============================] - 24s 54ms/step
[conf_1_jul] Processing ATs
Layer: dense_2
conf_1_jul ATs is saved at ./sa_mnist\model_31_jul_conf_1_jul_dense_2_ats.npy
yes
dict_keys([5, 7, 1, 2, 8, 3, 4, 9, 6, 0])
The number of removed columns: 23
[conf_1_jul] Fetching LSA
Duration: 0:20:19.111327


In [37]:
pwd

'D:\\deepimportance_code_release'

In [38]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
np.save(save_dir,np.array(target_lsa))

In [ ]:
# editing here

In [39]:
start_time = datetime.now()

target_dsa = sa.fetch_dsa(model, x_train, x_train_and_adversary,
                                   "conf_1_jul", layer_names,
                                   num_classes, True,
                                   './sa_'+dataset,  "model_31_jul")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Found saved train ATs, skip serving
[conf_1_jul] Model serving
8
438/438 [==============================] - 20s 45ms/step
[conf_1_jul] Processing ATs
Layer: dense_2
conf_1_jul ATs is saved at ./sa_mnist\model_31_jul_conf_1_jul_dense_2_ats.npy
[conf_1_jul] Fetching DSA
Duration: 0:47:25.128032


In [40]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"
np.save(save_dir,np.array(target_dsa))

In [42]:
#loading
loading =False
if (loading == True):
    save_dir = "C:/Users/fjdurlop/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
    target_lsa = np.load(save_dir)

    save_dir = "C:/Users/fjdurlop/Documents/upc-july/data/"+dataset+"/dsa_values.npy"
    target_dsa = np.load(save_dir)

In [43]:
print(target_dsa[48980:49010])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4198453e-07, 0.103650354, 0.110504985, 0.11916372, 0.09978015, 0.15995812, 0.10416289, 0.13666278, 0.08570866, 0.18017817, 0.12767863, 0.11186311]


In [44]:
print(target_dsa[49180:49210])

[0.16274497, 0.101583555, 0.12083733, 0.14051229, 0.11820864, 0.112263255, 0.085314415, 0.098417014, 0.13010219, 0.11316547, 0.11753961, 0.10477573, 0.101970434, 0.14521681, 0.110462874, 0.14807995, 0.12886639, 0.12255348, 0.11675078, 0.10050738, 0.114456624, 0.14356047, 0.14551164, 0.15918754, 0.0727881, 0.13202144, 0.11643102, 0.20601466, 0.11816457, 0.13781092]


In [45]:
print(target_dsa[40180:40210])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Random

In [46]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

[33012, 43384, 4051, 53430, 21307, 20002, 17585, 19801, 52822, 1005]
55998


In [48]:

save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"

np.save(save_dir,np.array(random_indexes))

In [26]:
cd D:/backup_UPC/upc/project_notebooks/TestSelection

D:\backup_UPC\upc\project_notebooks\TestSelection


In [27]:
cd ..

D:\backup_UPC\upc\project_notebooks


In [28]:
pwd

'D:\\backup_UPC\\upc\\project_notebooks'

In [29]:
import TestSelection
from TestSelection.utils.ncoverage import NCoverage # https://github.com/TestSelection/TestSelection

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [31]:
ncComputor_model_1 = NCoverage(model, threshold=0.2,only_layers=[ 'dense_2']) #last hidden layer

models loaded
['dense_2']


In [32]:
# number of total inputs
print(x_train_and_adversary.shape)

(55998, 28, 28, 1)


In [27]:
import time
start_time = time.time()
nc_model_original_values_1 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[:3000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 1610.7066071033478 seconds ---


In [28]:
start_time = time.time()
nc_model_original_values_2 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[3000:6000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 1605.5158166885376 seconds ---


In [29]:
x_train_and_adversary.shape

(55998, 28, 28, 1)

In [30]:
nc_model_original_values_3 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[6000:9000]]

In [31]:
nc_values = [
    nc_model_original_values_1,
    nc_model_original_values_2,
    nc_model_original_values_3
]

In [32]:
cd C:/Users/fjdur/Documents/upc-july/data/mnist


C:\Users\fjdur\Documents\upc-july\data\mnist


In [33]:
#Save nc values
i=1
for values in nc_values:
    with open(dataset+'_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [34]:
nc_model_original_values_4 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[9000:12000]]
nc_model_original_values_5 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[12000:15000]]
nc_model_original_values_6 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[15000:18000]]
nc_model_original_values_7 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[18000:21000]]

In [35]:
nc_values = [
    nc_model_original_values_4,
    nc_model_original_values_5,
    nc_model_original_values_6,
    nc_model_original_values_7
]

In [36]:
#Save nc values
i=4
for values in nc_values:
    with open('mnist_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [37]:
start_time = time.time()
nc_model_original_values_8 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[21000:24000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_9 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[24000:27000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_10 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[27000:30000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 1612.4683327674866 seconds ---
--- 3226.1769349575043 seconds ---
--- 4052.6816051006317 seconds ---


In [38]:
nc_values = [
    nc_model_original_values_8,
    nc_model_original_values_9,
    nc_model_original_values_10
]

In [39]:
#Save nc values
i=8
for values in nc_values:
    with open('mnist_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [33]:
start_time = time.time()

nc_model_original_values_11 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[30000:33000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_12 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[33000:36000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_13 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[36000:39000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 733.6739954948425 seconds ---
--- 1466.532585144043 seconds ---
--- 2202.209897994995 seconds ---


In [34]:
nc_values = [
    nc_model_original_values_11,
    nc_model_original_values_12,
    nc_model_original_values_13
]

In [35]:
#Save nc values
i=11
for values in nc_values:
    with open('mnist_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [36]:
start_time = time.time()

nc_model_original_values_14 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[39000:42000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_15 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[42000:45000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_16 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[45000:48000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 742.6888751983643 seconds ---
--- 1486.6844155788422 seconds ---
--- 2227.0559294223785 seconds ---


In [37]:
x_train_and_adversary.shape

(55998, 28, 28, 1)

In [38]:
nc_values = [
    nc_model_original_values_14,
    nc_model_original_values_15,
    nc_model_original_values_16
]

In [39]:
#Save nc values
i=14
for values in nc_values:
    with open('mnist_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [ ]:
#735 segs per 3000 inputs

In [40]:
nc_model_original_values_17 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[48000:51000]]
nc_model_original_values_18 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[51000:54000]]
nc_model_original_values_19 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[54000:]]

In [41]:
nc_values = [
    nc_model_original_values_17,
    nc_model_original_values_18,
    nc_model_original_values_19
]

In [42]:
#Save nc values
i=17
for values in nc_values:
    with open('mnist_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [43]:
pwd

'D:\\backup_UPC\\upc\\project_notebooks'

In [33]:
len(nc_model_original_values_18)

4999

## Uncertainty

In [58]:
from tensorflow.keras import layers
from tensorflow import keras

In [60]:
# Guardar el Modelo
model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_mnist.h5"

model.save(model_dir)



In [61]:
model.evaluate(x_test,y_test)

219/219 [==============================] - 2s 11ms/step - loss: 0.0366 - accuracy: 0.9894


[0.036614201962947845, 0.9894300699234009]

In [62]:
model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_mnist.h5"

model_original_functional = keras.models.load_model(model_dir)

In [63]:
model_original_functional.evaluate(x_test,y_test)

219/219 [==============================] - 3s 11ms/step - loss: 0.0366 - accuracy: 0.9894


[0.036614201962947845, 0.9894300699234009]

In [64]:
try:
  import uncertainty_wizard as uwiz
except ModuleNotFoundError as e:
  # Uncertainty wizard was not installed. Install it now (we're probably on colab)
  !pip install uncertainty_wizard
  import uncertainty_wizard as uwiz

In [65]:
type(model_original_functional)

tensorflow.python.keras.engine.sequential.Sequential

In [67]:
stochastic_model2 = uwiz.models.stochastic_from_keras(model_original_functional)


In [68]:
quantifiers = ['deep_gini']
results_1 = stochastic_model2.predict_quantified(x_train_and_adversary,
                                   quantifier=quantifiers,
                                   batch_size=64,
                                   sample_size=32,
                                   verbose=1)

875/875 [==============================] - 17s 19ms/step


In [69]:
quantifiers = [ 'softmax_entropy']
results_2 = stochastic_model2.predict_quantified(x_train_and_adversary,
                                   quantifier=quantifiers,
                                   batch_size=64,
                                   sample_size=32,
                                   verbose=1)

875/875 [==============================] - 20s 23ms/step


In [46]:
len(x_train_and_adversary)

55999

In [70]:
#deep_gini_values = results[0][1]
#softmax_values = results[1][1]

deep_gini_values = results_1[0][1]
softmax_values = results_2[0][1]

In [72]:
print(deep_gini_values.shape)
print(softmax_values.shape)

(55998,)
(55998,)


In [73]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"
np.save(save_dir,np.array(deep_gini_values))

save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"
np.save(save_dir,np.array(softmax_values))

In [74]:
softmax_values[-15:]

array([0.16717713, 0.40944645, 2.3119917 , 0.21343209, 0.6736572 ,
       0.0310397 , 0.6719951 , 1.2669243 , 0.01966682, 1.9966614 ,
       0.97211266, 0.07036015, 0.98849934, 0.64909196, 1.377052  ],
      dtype=float32)

In [75]:
softmax_values[:15]

array([1.5463616e-04, 1.9838268e-01, 1.0098722e-03, 6.1843824e-01,
       1.5649363e-02, 3.0791519e-02, 1.7598845e-02, 2.3867244e-05,
       4.7011952e-07, 7.2639450e-06, 2.4672965e-05, 2.4710948e-07,
       2.8708859e-05, 7.4917531e-01, 1.1955755e-02], dtype=float32)

In [76]:
y_test_and_adversary[:10]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
# results[0][0] contains the point-predictions from the 'pcs' quantifier
#     (i.e., argmax of a single non-randomized network forward pass)
# results[0][1] contains the prediction confidence scores for these predictions

# results[1][0] contains the predictions from the 'mean_softmax' quantifier
#     (i.e., argmax of 32 averages forward pass samples on the randomized DNN)
# results[1][1] contains the corresponding confidence
#     (i.e., the average softmax value of the class with the highest average softmax value)

In [30]:
print(results[0][0])

[7 9 2 ... 8 4 9]


In [78]:
print(results_2[0][1][:15])
print(results_2[0][1][-15:])

[1.5463616e-04 1.9838268e-01 1.0098722e-03 6.1843824e-01 1.5649363e-02
 3.0791519e-02 1.7598845e-02 2.3867244e-05 4.7011952e-07 7.2639450e-06
 2.4672965e-05 2.4710948e-07 2.8708859e-05 7.4917531e-01 1.1955755e-02]
[0.16717713 0.40944645 2.3119917  0.21343209 0.6736572  0.0310397
 0.6719951  1.2669243  0.01966682 1.9966614  0.97211266 0.07036015
 0.98849934 0.64909196 1.377052  ]


In [80]:
print(results_1[0][1][:15])
print(results_1[0][1][-15:])

[1.5974045e-05 5.4982126e-02 1.1563301e-04 2.0268744e-01 2.4631619e-03
 6.1262846e-03 3.2230616e-03 2.1457672e-06 0.0000000e+00 4.7683716e-07
 2.1457672e-06 0.0000000e+00 2.6226044e-06 2.3311919e-01 2.0952225e-03]
[0.03549308 0.10513455 0.7360556  0.04261881 0.16043854 0.0056935
 0.2402224  0.34408778 0.00295168 0.5682963  0.45457488 0.01497811
 0.43309665 0.17329419 0.4262073 ]
